In [7]:
import warnings
warnings.filterwarnings('ignore')

import os

PROJ_ROOT = os.pardir

TRAIN_DATA = os.path.join(PROJ_ROOT, "data", "raw", "train.csv")
TEST_DATA = os.path.join(PROJ_ROOT, "data", "raw", "test.csv")

import sys
sys.path.append(os.path.join(PROJ_ROOT, "src"))

%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [8]:
# from fastai.imports import *
# from fastai.structured import *

# from pandas_summary import DataFrameSummary
from IPython.display import display

import pandas as pd
import numpy as np

# from fastai.imports import *
# from fastai.torch_imports import *
# from fastai.transforms import *
# from fastai.conv_learner import *
# from fastai.model import *
# from fastai.dataset import *
# from fastai.sgdr import *
# from fastai.plots import *

In [9]:
PATH = os.path.join(PROJ_ROOT, "data", "raw/")

PATH


from sklearn.metrics import accuracy_score, r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [10]:
!ls {PATH} 

bodyfat-test.csv             kaggle_solution_template.csv
bodyfat-train.csv


In [11]:
train_df = pd.read_csv(f"{PATH}/bodyfat-train.csv")
test_df = pd.read_csv(f"{PATH}/bodyfat-test.csv")

In [12]:
train_df.describe()

,Body fat (%),Age (years),Weight (lbs),Height (inches),Neck (cm),Chest (cm),Abdomen (cm),Hip (cm),Thigh (cm),Knee (cm),Ankle (cm),Biceps (cm),Forearm (cm),Wrist (cm)
count,201.000000,201.000000,201.000000,201.000000,201.000000,201.000000,201.000000,201.000000,201.000000,201.000000,201.000000,201.000000,201.000000,201.000000
mean,18.977612,44.631841,178.370398,70.009950,37.992537,100.500000,92.392537,99.885572,59.407960,38.574627,23.060697,32.197015,28.645771,18.216418
std,8.391314,12.462896,30.489848,3.859877,2.549803,8.508314,10.973468,7.399307,5.503221,2.481875,1.647027,3.196043,2.080167,0.935189
min,0.000000,22.000000,118.500000,29.500000,31.100000,79.300000,69.400000,85.000000,47.200000,33.000000,19.100000,24.800000,22.000000,15.800000
25%,12.500000,35.000000,157.000000,68.250000,36.300000,94.000000,84.400000,95.500000,56.000000,36.900000,22.000000,30.000000,27.200000,17.600000
50%,19.200000,43.000000,176.000000,70.000000,38.000000,99.600000,90.900000,99.300000,58.900000,38.600000,22.800000,31.900000,28.700000,18.300000
75%,24.700000,54.000000,198.000000,72.250000,39.600000,105.300000,99.100000,103.700000,63.100000,39.900000,24.000000,34.600000,30.100000,18.800000
max,47.500000,81.000000,363.150000,77.500000,51.200000,136.200000,148.100000,147.700000,87.300000,49.100000,33.900000,45.000000,34.900000,21.400000


In [13]:
train_df_orig = train_df.copy()

In [14]:
train_df.corr()["Body fat (%)"]

Body fat (%)       1.000000
Age (years)        0.281098
Weight (lbs)       0.604166
Height (inches)   -0.123521
Neck (cm)          0.487892
Chest (cm)         0.712247
Abdomen (cm)       0.812182
Hip (cm)           0.628160
Thigh (cm)         0.560525
Knee (cm)          0.493188
Ankle (cm)         0.269730
Biceps (cm)        0.521717
Forearm (cm)       0.400249
Wrist (cm)         0.366006
Name: Body fat (%), dtype: float64

In [15]:
len(train_df)

201

In [16]:
train_df = train_df_orig.copy()

train_df['height_cm'] = train_df['Height (inches)']*2.54

In [17]:
def create_ratio(df, col1, col2):
    col_name = f"{col1}_{col2}_ratio"
    df[col_name] = df[col1]/df[col2]
    return col_name

In [18]:
ratio_list = [('height_cm',"Weight (lbs)"),
("Chest (cm)","Hip (cm)"),
("Age (years)","Weight (lbs)"),
("Knee (cm)","Thigh (cm)"),
("Chest (cm)","Abdomen (cm)")]

ratio_cols = []
for col1, col2 in ratio_list: 
    ratio_cols.append(create_ratio(train_df, col1,col2))
ratio_cols

['height_cm_Weight (lbs)_ratio',
 'Chest (cm)_Hip (cm)_ratio',
 'Age (years)_Weight (lbs)_ratio',
 'Knee (cm)_Thigh (cm)_ratio',
 'Chest (cm)_Abdomen (cm)_ratio']

In [19]:
columns_to_train = ['height_cm', "Age (years)", "Abdomen (cm)", "Neck (cm)"]
columns_to_train += ratio_cols
#columns_to_train += ['lbm']
columns_to_train

['height_cm',
 'Age (years)',
 'Abdomen (cm)',
 'Neck (cm)',
 'height_cm_Weight (lbs)_ratio',
 'Chest (cm)_Hip (cm)_ratio',
 'Age (years)_Weight (lbs)_ratio',
 'Knee (cm)_Thigh (cm)_ratio',
 'Chest (cm)_Abdomen (cm)_ratio']

In [20]:
train_df.corr()["Body fat (%)"]

Body fat (%)                      1.000000
Age (years)                       0.281098
Weight (lbs)                      0.604166
Height (inches)                  -0.123521
Neck (cm)                         0.487892
Chest (cm)                        0.712247
Abdomen (cm)                      0.812182
Hip (cm)                          0.628160
Thigh (cm)                        0.560525
Knee (cm)                         0.493188
Ankle (cm)                        0.269730
Biceps (cm)                       0.521717
Forearm (cm)                      0.400249
Wrist (cm)                        0.366006
height_cm                        -0.123521
height_cm_Weight (lbs)_ratio     -0.699267
Chest (cm)_Hip (cm)_ratio         0.312930
Age (years)_Weight (lbs)_ratio   -0.073027
Knee (cm)_Thigh (cm)_ratio       -0.368292
Chest (cm)_Abdomen (cm)_ratio    -0.690165
Name: Body fat (%), dtype: float64

In [21]:
train_df.columns

Index(['Body fat (%)', 'Age (years)', 'Weight (lbs)', 'Height (inches)',
       'Neck (cm)', 'Chest (cm)', 'Abdomen (cm)', 'Hip (cm)', 'Thigh (cm)',
       'Knee (cm)', 'Ankle (cm)', 'Biceps (cm)', 'Forearm (cm)', 'Wrist (cm)',
       'height_cm', 'height_cm_Weight (lbs)_ratio',
       'Chest (cm)_Hip (cm)_ratio', 'Age (years)_Weight (lbs)_ratio',
       'Knee (cm)_Thigh (cm)_ratio', 'Chest (cm)_Abdomen (cm)_ratio'],
      dtype='object')

In [23]:


from matplotlib import pyplot as plt
import seaborn as sns
def plot_feature(col_name):
    print(f"===========>{col_name}")
    plt.figure()
    sns.boxplot(x=train_df[col_name])

#     plt.plot(train_df[col_name])
    plt.show()
    plt.clf()
    plt.close()
    print(train_df[col_name].describe())
    return None

In [24]:
#plot_feature('Abdomen (cm)')

In [25]:
# for feature in train_df.columns:
#     plot_feature(feature)

In [26]:
print(f"mean max min max_count min_count")
for col in train_df.columns:
    max_val = np.max(train_df[col])
    min_val = np.min(train_df[col])
    avg = np.mean(train_df[col])
    max_count = len(train_df[train_df[col]>((max_val - avg)/2)])
    min_count = len(train_df[train_df[col]>((avg-min_val)/2)])
    print(f"{avg} \t {max_val} \t {min_val} \t {max_count} \t {min_count} \t {col} ")
    

mean max min max_count min_count
18.97761194029852 	 47.5 	 0.0 	 138 	 169 	 Body fat (%) 
44.6318407960199 	 81 	 22 	 201 	 201 	 Age (years) 
178.37039800995026 	 363.15 	 118.5 	 201 	 201 	 Weight (lbs) 
70.00995024875621 	 77.5 	 29.5 	 201 	 201 	 Height (inches) 
37.992537313432805 	 51.2 	 31.1 	 201 	 201 	 Neck (cm) 
100.50000000000001 	 136.2 	 79.3 	 201 	 201 	 Chest (cm) 
92.39253731343281 	 148.1 	 69.4 	 201 	 201 	 Abdomen (cm) 
99.8855721393035 	 147.7 	 85.0 	 201 	 201 	 Hip (cm) 
59.40796019900498 	 87.3 	 47.2 	 201 	 201 	 Thigh (cm) 
38.57462686567165 	 49.1 	 33.0 	 201 	 201 	 Knee (cm) 
23.06069651741294 	 33.9 	 19.1 	 201 	 201 	 Ankle (cm) 
32.19701492537314 	 45.0 	 24.8 	 201 	 201 	 Biceps (cm) 
28.64577114427861 	 34.9 	 22.0 	 201 	 201 	 Forearm (cm) 
18.21641791044776 	 21.4 	 15.8 	 201 	 201 	 Wrist (cm) 
177.82527363184082 	 196.85 	 74.93 	 201 	 201 	 height_cm 
1.0210788203897838 	 1.4575527426160337 	 0.36551219512195127 	 201 	 201 	 heigh

In [27]:
from lightgbm.sklearn import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import ExtraTreesRegressor, GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor, RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso

In [5]:
models = [RandomForestRegressor, GradientBoostingRegressor, LGBMRegressor, XGBRegressor, ExtraTreesRegressor, AdaBoostRegressor,
          BaggingRegressor, Ridge, Lasso]

In [86]:
target = train_df["Body fat (%)"]
train_X, test_X, train_y, test_y = train_test_split(train_df[columns_to_train], target, test_size = 0.2, random_state = 43)



In [109]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error as mas
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import make_scorer

def get_grid_best_estimator(X_train, y_train,base_estimator, params, scorer = None):
    # inputs of Grid search 
    
    # create cross validation sets 
    cv_sets = ShuffleSplit(n_splits = 10, test_size = 0.2, random_state = 50)
    #cv_sets = StratifiedKFold(n_splits = 10, shuffle = False, random_state = 50)
    
    # Default scorer is mean squared error
    if scorer is None: 
        scorer = make_scorer(mean_absolute_error,greater_is_better=False)
    
    # params
    
    # Grid search
    grid = GridSearchCV(base_estimator, params, scoring = scorer, cv = cv_sets )
    
    # fit 
    grid.fit(X_train,y_train)
    
    best_estimator  = grid.best_estimator_
    
    
    print("Best estimator : ")
    print(best_estimator)
    
    #print("CV_results:")
    #print(grid.cv_results_)
    
    return best_estimator



def train_and_test_grid_search(X_train, X_test, y_train, y_test, base_estimator, params, scorer = None):
    
    
    best_estimator = get_grid_best_estimator(X_train, y_train, base_estimator, params, scorer)
    
    print("Predicting the target for %d test samples:" %(len(X_test)))
    y_pred = best_estimator.predict(X_test)
    
    print("Prediction complete!")
    
    r2 = r2_score(y_pred, y_test)
    mas = mean_absolute_error(y_pred, y_test)
    print(f"r2: {r2} \t mas: {mas} \t Model {best_estimator}")
    return y_pred

In [88]:
from scipy import stats

def remove_outliers(features_df,target_df,z_score=2.75):
    X = features_df
    y = target_df
    if len(X) != len(y):
        print(f"Error: Number of samples in feature and target dataframe are not same : {len(X)} and {len(y)} ")
        return
    mask = (np.abs(stats.zscore(X)) < z_score).all(axis=1)
    print(f"Removed {len(X) - len(X[mask])} outlier samples out of {len(X)} from feature and target dataframes")
    return X[mask],y[mask]

train_X_or, train_y_or = remove_outliers(train_X, train_y)

Removed 9 outlier samples out of 160 from feature and target dataframes


In [89]:
len(train_X_or), len(train_X), len(train_y_or), len(train_y)
#train_X = train_X_or

(151, 160, 151, 160)

In [80]:
# train_X = train_X_or
# train_y = train_y_or

In [81]:
d = {}
model = None
for model in models:
    print(f"Training {model}")
    d[model] = model()
    d[model].fit(train_X, train_y)
    pred = d[model].predict(test_X)
    r2 = r2_score(pred, test_y)
    mas = mean_absolute_error(pred, test_y)
    print(f"r2: {r2} \t mas: {mas} \t Model {model}")
    d[model] = [d[model], r2, mas]
    

Training <class 'sklearn.ensemble.forest.RandomForestRegressor'>
r2: 0.257374769505081 	 mas: 4.199268292682927 	 Model <class 'sklearn.ensemble.forest.RandomForestRegressor'>
Training <class 'sklearn.ensemble.gradient_boosting.GradientBoostingRegressor'>
r2: 0.3393475371272141 	 mas: 4.190621566244524 	 Model <class 'sklearn.ensemble.gradient_boosting.GradientBoostingRegressor'>
Training <class 'lightgbm.sklearn.LGBMRegressor'>
r2: 0.5073426208989071 	 mas: 3.7017729318445793 	 Model <class 'lightgbm.sklearn.LGBMRegressor'>
Training <class 'xgboost.sklearn.XGBRegressor'>
r2: 0.3340110040133133 	 mas: 4.073500921668076 	 Model <class 'xgboost.sklearn.XGBRegressor'>
Training <class 'sklearn.ensemble.forest.ExtraTreesRegressor'>
r2: 0.374110716598435 	 mas: 4.110975609756099 	 Model <class 'sklearn.ensemble.forest.ExtraTreesRegressor'>
Training <class 'sklearn.ensemble.weight_boosting.AdaBoostRegressor'>
r2: 0.36985931554857443 	 mas: 3.792156087276323 	 Model <class 'sklearn.ensemble.we

In [108]:
# rf_params 
params = {'max_depth':[2,3,4,5,10],
         'n_estimators':[50,100,200,300],
         'booster':['gbtree','gblinear','dart']}



train_and_test_grid_search(train_X_or, test_X, train_y_or, test_y,XGBRegressor(),params )


Best estimator : 
XGBRegressor(base_score=0.5, booster='dart', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=2,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)
Predicting the target for 41 test samples:
Prediction complete!
r2: 0.25721912024836824 	 mas: 4.105300312507443 	 Model XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)


array([24.530586, 19.212473, 21.402187, 23.58047 , 17.585123, 13.708346,
        9.524056, 20.21762 , 22.159826, 33.14605 , 11.357574, 23.329897,
       24.113998,  8.101816, 25.802692, 18.261665, 11.056096, 24.180044,
       17.91078 , 23.738026, 14.206358, 21.89691 ,  9.452186, 15.397398,
        8.200375, 22.743301, 21.170483, 24.336874, 17.79172 , 19.125803,
       22.97938 , 21.97662 , 24.357445, 15.774287, 19.480745, 27.067154,
       10.456526, 18.04754 , 21.737772, 19.715822, 18.917862],
      dtype=float32)

In [110]:
def get_xgb():
    return XGBRegressor(base_score=0.5, booster='dart', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=2,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [115]:
# Adaboost
from sklearn.tree import DecisionTreeRegressor

params = {'base_estimator':[DecisionTreeRegressor(max_depth=3),DecisionTreeRegressor(max_depth=10)],
          'n_estimators':[10,25,50,100,200, 500],
         'learning_rate':[1,0.1, 0.001],
         'loss':['linear','square','exponential']}



train_and_test_grid_search(train_X_or, test_X, train_y_or, test_y,AdaBoostRegressor(),params )


Best estimator : 
AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mse', max_depth=3, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
         learning_rate=1, loss='exponential', n_estimators=25,
         random_state=None)
Predicting the target for 41 test samples:
Prediction complete!
r2: 0.23634037786497908 	 mas: 4.110026427351787 	 Model AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mse', max_depth=3, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
         learning_rate=1, loss='exponential', n_estimators=25,
         random_sta

array([22.54418605, 19.704     , 20.37534247, 27.5       , 17.80571429,
       13.27692308, 10.1375    , 20.37534247, 25.24      , 28.72307692,
       10.68064516, 20.89726027, 20.43690476, 10.425     , 26.20447761,
       19.67912088, 13.23636364, 27.0195122 , 20.43690476, 25.42916667,
       17.80571429, 21.53283582, 10.1375    , 19.12272727,  8.232     ,
       22.54418605, 20.59152542, 26.01764706, 19.97291667, 17.4       ,
       26.68461538, 25.42916667, 22.775     , 16.576     , 20.07878788,
       27.525     ,  9.885     , 19.67912088, 21.53283582, 20.87727273,
       20.07878788])

In [120]:
params = {'alpha':[1,0.9, 0.1, 0.11, 0.2,0.5,0,0.001,0.75]}



train_and_test_grid_search(train_X_or, test_X, train_y_or, test_y,Lasso(),params )


Best estimator : 
Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)
Predicting the target for 41 test samples:
Prediction complete!
r2: 0.5891409616369019 	 mas: 3.7255836660172097 	 Model Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)


/Users/vthotakura/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:528: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
/Users/vthotakura/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:478: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
/Users/vthotakura/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/vthotakura/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:528: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimato

array([23.51756256, 16.70108654, 18.44502267, 24.6416983 , 14.11615401,
       16.2219814 ,  8.67344508, 18.23348352, 23.1148111 , 49.10190023,
        6.23757515, 24.00297988, 21.93946268,  9.67945975, 27.89618311,
       14.69463638, 12.13713507, 29.92515375, 18.76194408, 27.90916513,
       14.81497048, 21.40596509,  9.04553262, 14.99997503,  7.15086651,
       20.01908659, 22.14088155, 26.63620156, 17.33225428, 18.63673513,
       23.16154635, 22.68854226, 25.48491537, 16.94114116, 16.8093846 ,
       27.01295043, 11.47031637, 16.81179551, 19.96403095, 19.72808195,
       19.74896792])

In [123]:
params = {'alpha':[1,0.9, 0.1, 0.11, 0.2,0.5,0,0.001,0.75],
         'solver' : ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']}



train_and_test_grid_search(train_X_or, test_X, train_y_or, test_y,Ridge(),params )


Best estimator : 
Ridge(alpha=0.75, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='sparse_cg', tol=0.001)
Predicting the target for 41 test samples:
Prediction complete!
r2: 0.5901724421131652 	 mas: 3.7359687227241847 	 Model Ridge(alpha=0.75, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='sparse_cg', tol=0.001)


array([23.49295563, 16.77955221, 18.5459369 , 24.78869424, 14.0603507 ,
       16.26775477,  8.50474034, 18.28769951, 23.11077337, 49.14018731,
        6.21429059, 23.9923918 , 22.12023352,  9.59439276, 27.85645275,
       14.52860369, 12.00436943, 29.89668125, 18.72793744, 27.9703501 ,
       14.82753275, 21.33105538,  8.90914396, 14.93376077,  7.15404201,
       19.9334127 , 22.04300021, 26.8921058 , 17.29035214, 18.69442096,
       23.15393241, 22.53813683, 25.51496405, 16.86312175, 16.78840195,
       27.08733406, 11.62762954, 16.77812599, 19.83857859, 19.7599021 ,
       19.74679431])

In [ ]:
from lightgbm.sklearn import LGBMModel

random_state = 42

def get_lgbm_obj():
    return LGBMRegressor( objective = 'regression',
                        scale_pos_weight = 1,
                          metric = "auc", 
                          boosting = 'gbdt', 
                          max_depth = -1, 
                          num_leaves = 13, 
                          learning_rate = 0.01, 
                          bagging_freq = 5, 
                          bagging_fraction = 0.4, 
                          feature_fraction = 0.05, 
                          min_data_in_leaf = 80, 
                          min_sum_heassian_in_leaf = 10, 
                          tree_learner = "serial", 
                          boost_from_average = False, 
                          bagging_seed = random_state, 
                          verbosity = 1, 
                          seed = random_state,
                         num_boost_round = 100000,
                         is_training_metric = True,
                         is_provide_training_metric=True)

def get_lgbm_obj2():
    return LGBMRegressor( objective = 'regression',
                          metric = "auc", 
                          seed = random_state,
                         num_boost_round = 100000,
                         is_training_metric = True,
                         is_provide_training_metric=True)
                          
                          

In [ ]:
lgbm = get_lgbm_obj2()

In [ ]:
lgbm.fit(train_X, train_y)

In [ ]:
pred = lgbm.predict(test_X)

In [ ]:
r2 = r2_score(pred, test_y)
mas = mean_absolute_error(pred, test_y)
print(f"Model {model}, r2: {r2}, mas: {mas}")
